## Example showing generalized processing of commuting terms

## Extract Qubit-wise Commuting Groups from a Hamiltonian -  Working Example 


In [62]:
from qiskit.quantum_info import Pauli, SparsePauliOp
from itertools import combinations
import numpy as np
from math import sin, cos, pi

# Observable Helper Functions
from observables import *
    
# Example usage
hamiltonian = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

''' another test
H = SparsePauliOp(['ZZII', 'ZIIZ', 'IZZI', 'IIZZ', 'XIII', 'IXII', 'IIIX', 'IIXI'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 2.+0.j, 2.+0.j, 2.+0.j, 2.+0.j])
hamiltonian = [
    ('ZZII', 0.5),
    ('ZIIZ', 0.3),
    ('IZZI', 0.4),
    ('IIZZ', 0.2),
    ('XIII', 0.6),
    ('IXII', 0.1),
    ('IIIX', 0.7),
    ('IIXI', 0.7)
]
'''

groups = group_commuting_terms_2(hamiltonian)
for i, group in enumerate(groups):
    print(f"Group {i+1}:")
    for pauli, coeff in group:
        print(f"  {pauli}: {coeff}")



[SparsePauliOp(['XXII'],
              coeffs=[0.5+0.j]), SparsePauliOp(['IYYI'],
              coeffs=[0.3+0.j]), SparsePauliOp(['IIZZ'],
              coeffs=[0.4+0.j]), SparsePauliOp(['XYII'],
              coeffs=[0.2+0.j]), SparsePauliOp(['IIYX'],
              coeffs=[0.6+0.j]), SparsePauliOp(['IZXI'],
              coeffs=[0.1+0.j]), SparsePauliOp(['XIII'],
              coeffs=[0.7+0.j])]
[[False False  True False  True False  True]
 [False False False  True  True False  True]
 [ True False False  True False False  True]
 [False  True  True False  True False  True]
 [ True  True False  True False False  True]
 [False False False False False False  True]
 [ True  True  True  True  True  True False]]
    ... conflict, do not add to this group
    ... conflict, do not add to this group
    ... conflict, do not add to this group
    ... conflict, do not add to this group
    ... conflict, do not add to this group
Group 1:
  XXII: 0.5
  IIZZ: 0.4
  XIII: 0.7
Group 2:
  IYYI: 0.3
  X

## Compute the energy

In [71]:
from qiskit import QuantumCircuit,transpile, assemble
from qiskit_aer import Aer
import numpy as np

# Initialize the backend and the simulator
backend = Aer.get_backend('qasm_simulator')

'''
groups = [
    [('XXII', 0.5), ('XYII', 0.2), ('XIII', 0.7)],
    [('IYYI', 0.3), ('IIYX', 0.6)],
    [('IIZZ', 0.4), ('IZXI', 0.1)]
]

print("******** processing groups:")
print(groups)
'''



# Function to create circuits for raw Hamiltonian
def create_circuits_ham(ham):
    circuits = []

    for term, coeff in ham:
        print(f"  ... {term}, {coeff}")
        qc = QuantumCircuit(num_qubits)
        for i, pauli in enumerate(term):
            if pauli == 'X':
                qc.h(i)
            elif pauli == 'Y':
                qc.sdg(i)
                qc.h(i)
        qc.measure_all()
        circuits.append((qc, [(term, coeff)]))

    return circuits

# Function to create circuits for each group
def create_circuits(groups):
    circuits = []
    for group in groups:
        qc = QuantumCircuit(num_qubits)

        ## This is NOT correct, it is adding gates for each term; instead it needs to merge the terms and add one for each qubit
        '''
        for term, coeff in group:
            for i, pauli in enumerate(term):
                if pauli == 'X':
                    qc.h(i)
                elif pauli == 'Y':
                    qc.sdg(i)
                    qc.h(i)
         '''
        merged_paulis = ['I'] * num_qubits
        for term, coeff in group:
            for i, pauli in enumerate(term):
                if pauli != "I": merged_paulis[i] = pauli

        print(f"... merged_paulis = {merged_paulis}")

        merged_term = "".join(merged_paulis)
        print(f"... merged_term = {merged_term}")
            
        for i, pauli in enumerate(merged_term):
            if pauli == 'X':
                qc.h(i)
            elif pauli == 'Y':
                qc.sdg(i)
                qc.h(i)
                    
        qc.measure_all()
        circuits.append((qc, group))

    return circuits

# Function to calculate expectation values from results
# This function operates on tuples of (circuit, group)
def calculate_expectation(results, circuits, is_commuting=False):
    total_energy = 0
    for (qc, group), result in zip(circuits, results.get_counts()):
        #counts = result.get_counts()
        counts = result
        num_shots = sum(counts.values())
        print(f"... got num_shots = {num_shots}: counts = {counts}")

        if is_commuting:
            merge_terms(num_qubits, group)
            
            for term, coeff in group:
                print(f"--> for term: {term}, {coeff}")
                
                # Calculate the expectation value for each term
                exp_val = get_expectation_term(term, counts)  
                #exp_val /= num_shots
                
                total_energy += coeff * exp_val
                print(f"  ******* exp_val = {exp_val} {coeff * exp_val}")

        else:
            for term, coeff in group:
                print(f"--> for term: {term}, {coeff}")
                # Calculate the expectation value for each term
                exp_val = get_expectation_term(term, counts)
                '''
                exp_val = 0
                for bitstring, count in counts.items():
                    print(f"... {bitstring} {count}")
                    parity = (-1)**sum([int(bitstring[i]) for i, pauli in enumerate(term) if pauli != 'I'])
    
                    exp_val += parity * count
                    print(f"... parity = {parity} {exp_val}")
                '''  
                #exp_val /= num_shots
                
                total_energy += coeff * exp_val
                print(f"  ******* exp_val = {exp_val} {coeff * exp_val}")
            
    return total_energy

# Calculate the expectation value for each term

def get_expectation_term(term, counts):
    exp_val = 0
    total_counts = sum(counts.values())
    N = len(term)  # Total number of qubits

    for bitstring, count in counts.items():
        parity = 1.0  # Initialize parity for this bitstring

        for qubit_index, pauli in enumerate(term):
            if pauli != 'I':
                # Map qubit index to bitstring index (little-endian)
                bit_index = N - 1 - qubit_index
                bit_value = int(bitstring[bit_index])

                # Map bit_value to eigenvalue: 0 -> +1, 1 -> -1
                eigenvalue = 1 - 2 * bit_value  # 0 -> +1, 1 -> -1
                parity *= eigenvalue

        exp_val += parity * count

    # Normalize by total number of shots to get the expectation value
    return exp_val / total_counts

'''
def get_expectation_term(term, counts):
    exp_val = 0
    total_counts = sum(counts.values())

    for bitstring, count in counts.items():
        # Reverse the bitstring to match Qiskit’s little-endian convention
        bitstring = bitstring[::-1]
        
        # Start with a parity of +1 for each measurement result
        parity = 1.0
        
        # Loop over each qubit in the term and apply eigenvalue mapping
        for i, pauli in enumerate(term):
            if pauli != 'I':  # Only consider qubits with X, Y, or Z (ignore 'I')
                # Map 0 to +1, 1 to -1 based on the bit value for this qubit
                parity *= (-2 * int(bitstring[i]) + 1)
        
        # Accumulate weighted parity for this bitstring
        exp_val += parity * count
    
    # Normalize by total number of shots to get the expectation value
    return exp_val / total_counts
''' 
'''
def get_expectation_term(term, counts):
    exp_val = 0
    total_counts = sum(counts.values())

    for bitstring, count in counts.items():
        parity = 1.0  # Start with +1 for each measurement result
        for i, pauli in enumerate(term):
            if pauli != 'I':  # Only consider relevant qubits
                # Map 0 to +1, 1 to -1 based on the bit value
                parity *= (-2 * int(bitstring[i]) + 1)
        
        # Accumulate weighted parity for this bitstring
        exp_val += parity * count
    
    # Normalize by total number of shots to get the expectation value
    return exp_val / total_counts
'''
'''    
def get_expectation_term(term, counts):
    exp_val = 0
    total_counts = sum(counts.values())

    relevant_qubits = [i for i, pauli in enumerate(term) if pauli != 'I']
    print(f"... relevant = {relevant_qubits}")
    
    for bitstring, count in counts.items():
        print(f"... {bitstring} {count}")
        print([int(bitstring[i]) for i, pauli in enumerate(term) if pauli != 'I'])
        #parity = (-1)**sum([int(bitstring[i]) for i, pauli in enumerate(term) if pauli != 'I'])
        parity = (-1) ** sum(int(bitstring[i]) for i in relevant_qubits)
        
        print(f"... raw exp val {parity * count}")
        exp_val += parity * count
        print(f"  ... parity = {parity} {exp_val}")
        
    # Normalize by total number of shots to get the expectation value
    return exp_val / total_counts
'''
'''
def get_expectation_term(term, counts):
    # Identify the relevant qubits for the term
    relevant_qubits = [i for i, pauli in enumerate(term) if pauli != 'I']
    
    exp_val = 0
    total_counts = sum(counts.values())
    
    for bitstring, count in counts.items():
        print(f"... {bitstring} {count}")
        
        # Calculate the parity for only the relevant qubits
        parity = (-1) ** sum(int(bitstring[i]) for i in relevant_qubits)
        exp_val += parity * count

        print(f"... parity = {parity} {exp_val}")
    
    # Normalize by total number of shots to get the expectation value
    return exp_val / total_counts
'''
'''
def get_expectation_term(term, counts):
    # Find the indices of relevant qubits (non-'I' qubits in the term)
    relevant_qubits = [i for i, pauli in enumerate(term) if pauli != 'I']
    
    exp_val = 0
    total_counts = sum(counts.values())
    
    for bitstring, count in counts.items():
        # Calculate parity only for relevant qubits in the bitstring
        # Interpret `00` or `11` as +1 and `01` or `10` as -1
        relevant_bits = ''.join(bitstring[i] for i in relevant_qubits)
        parity = (-1) ** relevant_bits.count('1')  # Parity based on the number of '1's
        
        # Accumulate weighted contribution of parity * count for this term
        exp_val += parity * count
    
    # Normalize by the total number of shots
    return exp_val / total_counts
''' 

# Merge a group of commuting terms into a merged_term
def merge_terms(num_qubits, group):
    
    merged_paulis = ['I'] * num_qubits
    for term, coeff in group:
        for i, pauli in enumerate(term):
            if pauli != "I": merged_paulis[i] = pauli

    print(f"... merged_paulis = {merged_paulis}")

    merged_term = "".join(merged_paulis)
    print(f"... merged_term = {merged_term}")

    return merged_term



# ======================================================
'''
# Define the Hamiltonian terms
H_terms = [
    ('ZI', 0.5),
    ('XX', 0.3),
    ('YY', -0.1),
    #(-0.2, 'ZZ')
]

hamiltonian = H_terms
'''
# Example usage
hamiltonian = [
    ('XXII', 0.5),
    ('IYYI', 0.3),
    ('IIZZ', 0.4),
    ('XYII', 0.2),
    ('IIYX', 0.6),
    ('IZXI', 0.1),
    ('XIII', 0.7)
]

num_qubits = 4

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()
    
H = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
#H_terms = swap_pauli_list(H.to_list())
H_terms = H.to_list()

hamiltonian = H_terms

print(H)
print(hamiltonian)

groups = group_commuting_terms_2(hamiltonian)
for i, group in enumerate(groups):
    print(f"Group {i+1}:")
    for pauli, coeff in group:
        print(f"  {pauli}: {coeff}")


# optimize by use of commuting groups
use_commuting_groups = False
use_commuting_groups = True

# Create the circuits
if not use_commuting_groups:   
    print("\n******** creating circuits from Hamltonian:")
    print(hamiltonian)
    circuits = create_circuits_ham(hamiltonian)
    
else:   
    print("\n******** creating circuits from groups:")
    print(groups)
    circuits = create_circuits(groups)

# print the circuts
for circuit in circuits:
    print(f"\ncircuit = {circuit}")

print("************ compute energy N times for these groups:")

N = 1

# Calculate the total energy
for i in range(N):

    print("")
    
    # Compile and execute the circuits
    transpiled_circuits = transpile([circuit for circuit, group in circuits], backend)

    for tc in transpiled_circuits:
        print(tc)

    print("")
    
    #qobj = assemble(transpiled_circuits)
    results = backend.run(transpiled_circuits).result()

    total_energy = calculate_expectation(results, circuits, is_commuting=use_commuting_groups)
    print(f"Total Energy: {total_energy}")
    
    print("")


SparsePauliOp(['IIZZ', 'IZZI', 'ZZII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIIX', 'IIXI', 'IXII', 'XIII'],
              coeffs=[-0.2       +0.j, -0.2       +0.j, -0.2       +0.j, -0.45922012+0.j,
 -0.45922012+0.j, -0.45922012+0.j, -0.45922012+0.j, -1.10865544+0.j,
 -1.10865544+0.j, -1.10865544+0.j, -1.10865544+0.j])
[('IIZZ', (-0.2+0j)), ('IZZI', (-0.2+0j)), ('ZZII', (-0.2+0j)), ('IIIZ', (-0.4592201188381077+0j)), ('IIZI', (-0.4592201188381077+0j)), ('IZII', (-0.4592201188381077+0j)), ('ZIII', (-0.4592201188381077+0j)), ('IIIX', (-1.1086554390135441+0j)), ('IIXI', (-1.1086554390135441+0j)), ('IXII', (-1.1086554390135441+0j)), ('XIII', (-1.1086554390135441+0j))]
[SparsePauliOp(['IIZZ'],
              coeffs=[-0.2+0.j]), SparsePauliOp(['IZZI'],
              coeffs=[-0.2+0.j]), SparsePauliOp(['ZZII'],
              coeffs=[-0.2+0.j]), SparsePauliOp(['IIIZ'],
              coeffs=[-0.45922012+0.j]), SparsePauliOp(['IIZI'],
              coeffs=[-0.45922012+0.j]), SparsePauliOp(['IZII'],
   